In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
def get_rec(year):
    base_url = f"https://www.planecrashinfo.com/{year}"
    # Fetch the HTML content
    url = f"{base_url}/{year}"
    response = requests.get(f"{url}.htm")
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the first table
    table = soup.find("table")

    if table is None:
        return None

    # Extract headers
    headers = [td.text.strip() for td in table.find_all("tr")[0]]

    # Extract rows
    rows = []
    cnt = 0
    for tr in table.find_all("tr")[1:]:  # Skip header row
        cells = [td.text.strip() for td in tr.find_all("td")]
        if cells:
            rows.append(cells)
            cnt += 1

    # # Convert to DataFrame
    # df = pd.DataFrame(rows, columns=headers)

    # # Display DataFrame
    # df.head()
    ds = []
    for i in range(1, cnt+1):
        response = requests.get(url + f"-{i}.htm")
        soup = BeautifulSoup(response.text, "html.parser")

        # Find the first table
        table = soup.find("table")
        
        # Extract rows
        rows = []
        for tr in table.find_all("tr")[1:]:  # Skip header row
            cells = [td.text.strip() for td in tr.find_all("td")]
            if cells:
                rows.append(cells)

        ds.append(rows)

    scrapped_ds = [dict(rec) for rec in ds]
    return scrapped_ds
    
    



In [13]:
print(f'failed: {2024}')

failed: 2024


In [17]:
years = range(1920, 2026)

for yr in years:
    if get_rec(yr) is None:
        print(f'failed: {yr}')
    else:
        print(yr)
    
    

1920
1921
1922
1923
1924
1925


KeyboardInterrupt: 